In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import torchinfo
import my

In [2]:
df = pd.read_csv('./ai_job_market.csv')
df


,job_id,company_name,industry,job_title,skills_required,experience_level,employment_type,location,salary_range_usd,posted_date,company_size,tools_preferred
0,1,Foster and Sons,Healthcare,Data Analyst,"NumPy, Reinforcement Learning, PyTorch, Scikit...",Mid,Full-time,"Tracybury, AR",92860-109598,2025-08-20,Large,"KDB+, LangChain"
1,2,"Boyd, Myers and Ramirez",Tech,Computer Vision Engineer,"Scikit-learn, CUDA, SQL, Pandas",Senior,Full-time,"Lake Scott, CU",78523-144875,2024-03-22,Large,"FastAPI, KDB+, TensorFlow"
2,3,King Inc,Tech,Quant Researcher,"MLflow, FastAPI, Azure, PyTorch, SQL, GCP",Entry,Full-time,"East Paige, CM",124496-217204,2025-09-18,Large,"BigQuery, PyTorch, Scikit-learn"
3,4,"Cooper, Archer and Lynch",Tech,AI Product Manager,"Scikit-learn, C++, Pandas, LangChain, AWS, R",Mid,Full-time,"Perezview, FI",50908-123743,2024-05-08,Large,"TensorFlow, BigQuery, MLflow"
4,5,Hall LLC,Finance,Data Scientist,"Excel, Keras, SQL, Hugging Face",Senior,Contract,"North Desireeland, NE",98694-135413,2025-02-24,Large,"PyTorch, LangChain"
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,"Mueller, Ellis and Clark",Finance,NLP Engineer,"Flask, FastAPI, Power BI",Senior,Internship,"Washingtonmouth, SD",90382-110126,2024-04-22,Large,MLflow
1996,1997,Roberts-Yu,Automotive,AI Product Manager,"R, Flask, Excel, C++, CUDA, Scikit-learn",Mid,Remote,"Joshuafort, ZA",47848-137195,2023-12-02,Large,"KDB+, LangChain, MLflow"
1997,1998,"Brooks, Williams and Randolph",Education,Data Analyst,"Hugging Face, Excel, Scikit-learn, R, MLflow",Entry,Contract,"West Brittanyburgh, CG",134994-180108,2023-10-29,Large,PyTorch
1998,1999,Castaneda-Smith,Education,Quant Researcher,"AWS, Python, Scikit-learn",Senior,Contract,"Anthonyshire, OM",62388-82539,2024-08-10,Large,"MLflow, TensorFlow, FastAPI"


In [3]:

df['salary_range_usd']
df[['Min_Salary', 'Max_Salary']] = df['salary_range_usd'].str.split('-', expand=True)
df['Min_Salary'] = pd.to_numeric(df['Min_Salary'], errors='coerce')
df['Max_Salary'] = pd.to_numeric(df['Max_Salary'], errors='coerce')
df['Average_Salary']=((df["Min_Salary"]+df["Max_Salary"])/20000)


skills=df["skills_required"].str.split(',', expand=True)
skills.columns = [f'Skill_{i+1}' for i in range(skills.shape[1])]
tools=df["tools_preferred"].str.split(',', expand=True)
tools.columns = [f'Tools_{i+1}' for i in range(tools.shape[1])]
skills
df['Average_Salary']


0       10.12290
1       11.16990
2       17.08500
3        8.73255
4       11.70535
          ...   
1995    10.02540
1996     9.25215
1997    15.75510
1998     7.24635
1999     7.66045
Name: Average_Salary, Length: 2000, dtype: float64

In [4]:
df_combine={}
df = pd.concat([df, skills,tools], axis=1)
df


,job_id,company_name,industry,job_title,skills_required,experience_level,employment_type,location,salary_range_usd,posted_date,...,Average_Salary,Skill_1,Skill_2,Skill_3,Skill_4,Skill_5,Skill_6,Tools_1,Tools_2,Tools_3
0,1,Foster and Sons,Healthcare,Data Analyst,"NumPy, Reinforcement Learning, PyTorch, Scikit...",Mid,Full-time,"Tracybury, AR",92860-109598,2025-08-20,...,10.12290,NumPy,Reinforcement Learning,PyTorch,Scikit-learn,GCP,FastAPI,KDB+,LangChain,None
1,2,"Boyd, Myers and Ramirez",Tech,Computer Vision Engineer,"Scikit-learn, CUDA, SQL, Pandas",Senior,Full-time,"Lake Scott, CU",78523-144875,2024-03-22,...,11.16990,Scikit-learn,CUDA,SQL,Pandas,None,None,FastAPI,KDB+,TensorFlow
2,3,King Inc,Tech,Quant Researcher,"MLflow, FastAPI, Azure, PyTorch, SQL, GCP",Entry,Full-time,"East Paige, CM",124496-217204,2025-09-18,...,17.08500,MLflow,FastAPI,Azure,PyTorch,SQL,GCP,BigQuery,PyTorch,Scikit-learn
3,4,"Cooper, Archer and Lynch",Tech,AI Product Manager,"Scikit-learn, C++, Pandas, LangChain, AWS, R",Mid,Full-time,"Perezview, FI",50908-123743,2024-05-08,...,8.73255,Scikit-learn,C++,Pandas,LangChain,AWS,R,TensorFlow,BigQuery,MLflow
4,5,Hall LLC,Finance,Data Scientist,"Excel, Keras, SQL, Hugging Face",Senior,Contract,"North Desireeland, NE",98694-135413,2025-02-24,...,11.70535,Excel,Keras,SQL,Hugging Face,None,None,PyTorch,LangChain,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,"Mueller, Ellis and Clark",Finance,NLP Engineer,"Flask, FastAPI, Power BI",Senior,Internship,"Washingtonmouth, SD",90382-110126,2024-04-22,...,10.02540,Flask,FastAPI,Power BI,None,None,None,MLflow,None,None
1996,1997,Roberts-Yu,Automotive,AI Product Manager,"R, Flask, Excel, C++, CUDA, Scikit-learn",Mid,Remote,"Joshuafort, ZA",47848-137195,2023-12-02,...,9.25215,R,Flask,Excel,C++,CUDA,Scikit-learn,KDB+,LangChain,MLflow
1997,1998,"Brooks, Williams and Randolph",Education,Data Analyst,"Hugging Face, Excel, Scikit-learn, R, MLflow",Entry,Contract,"West Brittanyburgh, CG",134994-180108,2023-10-29,...,15.75510,Hugging Face,Excel,Scikit-learn,R,MLflow,None,PyTorch,None,None
1998,1999,Castaneda-Smith,Education,Quant Researcher,"AWS, Python, Scikit-learn",Senior,Contract,"Anthonyshire, OM",62388-82539,2024-08-10,...,7.24635,AWS,Python,Scikit-learn,None,None,None,MLflow,TensorFlow,FastAPI


In [20]:
label1, uniqueValues1= pd.factorize(df['Skill_1'])
label2, uniqueValues2= pd.factorize(df['Skill_2'])
label3, uniqueValues3= pd.factorize(df['Skill_3'])
label4, uniqueValues4= pd.factorize(df['Skill_4'])
label5, uniqueValues5= pd.factorize(df['Skill_5'])
label6, uniqueValues6= pd.factorize(df['Tools_1'])
label7, uniqueValues7= pd.factorize(df['Tools_2'])
label8, uniqueValues8= pd.factorize(df['Tools_3'])
label9, uniqueValues9= pd.factorize(df['location'])
label10, uniqueValues10= pd.factorize(df['employment_type'])
label11, uniqueValues11= pd.factorize(df['job_title'])
label12, uniqueValues12= pd.factorize(df['industry'])
label13, uniqueValues13= pd.factorize(df['company_name'])



In [21]:
targetlabel, targetunique= pd.factorize(df['Average_Salary'])
df['target']=targetlabel
df['target']
targetunique

Index([ 10.1229,  11.1699,   17.085,  8.73255, 11.70535,  13.6675,  7.95845,
         7.2588, 16.13715,  12.2195,
       ...
        16.5924,  11.4291,  5.49335,  19.1729,  10.7884,  10.0254,  9.25215,
        15.7551,  7.24635,  7.66045],
      dtype='float64', length=1992)

In [22]:
df['fskill_1']=label1
df['fskill_2']=label2
df['fskill_3']=label3
df['fskill_4']=label4
df['fskill_5']=label5
df['ftool_1']=label6
df['ftool_2']=label7
df['ftool_3']=label8
df['floc']=label9
df['ftype']=label10
df['ftitle']=label11
df['findustry']=label12
df['fname']=label13


In [23]:
df


,job_id,company_name,industry,job_title,skills_required,experience_level,employment_type,location,salary_range_usd,posted_date,...,fskill_4,fskill_5,ftool_1,ftool_2,ftool_3,floc,ftype,ftitle,findustry,fname
0,1,Foster and Sons,Healthcare,Data Analyst,"NumPy, Reinforcement Learning, PyTorch, Scikit...",Mid,Full-time,"Tracybury, AR",92860-109598,2025-08-20,...,0,0,0,0,-1,0,0,0,0,0
1,2,"Boyd, Myers and Ramirez",Tech,Computer Vision Engineer,"Scikit-learn, CUDA, SQL, Pandas",Senior,Full-time,"Lake Scott, CU",78523-144875,2024-03-22,...,1,-1,1,1,0,1,0,1,1,1
2,3,King Inc,Tech,Quant Researcher,"MLflow, FastAPI, Azure, PyTorch, SQL, GCP",Entry,Full-time,"East Paige, CM",124496-217204,2025-09-18,...,2,1,2,2,1,2,0,2,1,2
3,4,"Cooper, Archer and Lynch",Tech,AI Product Manager,"Scikit-learn, C++, Pandas, LangChain, AWS, R",Mid,Full-time,"Perezview, FI",50908-123743,2024-05-08,...,3,2,3,3,2,3,0,3,1,3
4,5,Hall LLC,Finance,Data Scientist,"Excel, Keras, SQL, Hugging Face",Senior,Contract,"North Desireeland, NE",98694-135413,2025-02-24,...,4,-1,4,0,-1,4,1,4,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,"Mueller, Ellis and Clark",Finance,NLP Engineer,"Flask, FastAPI, Power BI",Senior,Internship,"Washingtonmouth, SD",90382-110126,2024-04-22,...,-1,-1,6,-1,-1,1995,3,6,2,1904
1996,1997,Roberts-Yu,Automotive,AI Product Manager,"R, Flask, Excel, C++, CUDA, Scikit-learn",Mid,Remote,"Joshuafort, ZA",47848-137195,2023-12-02,...,13,13,0,0,2,1996,2,3,4,1905
1997,1998,"Brooks, Williams and Randolph",Education,Data Analyst,"Hugging Face, Excel, Scikit-learn, R, MLflow",Entry,Contract,"West Brittanyburgh, CG",134994-180108,2023-10-29,...,19,7,4,-1,-1,1997,1,0,5,1906
1998,1999,Castaneda-Smith,Education,Quant Researcher,"AWS, Python, Scikit-learn",Senior,Contract,"Anthonyshire, OM",62388-82539,2024-08-10,...,-1,-1,6,4,3,1998,1,2,5,1907


In [24]:

inputs = torch.tensor(df.iloc[:,-13:].values, dtype=torch.float32)
df.iloc[:,-13:]

,fskill_1,fskill_2,fskill_3,fskill_4,fskill_5,ftool_1,ftool_2,ftool_3,floc,ftype,ftitle,findustry,fname
0,0,0,0,0,0,0,0,-1,0,0,0,0,0
1,1,1,1,1,-1,1,1,0,1,0,1,1,1
2,2,2,2,2,1,2,2,1,2,0,2,1,2
3,1,3,3,3,2,3,3,2,3,0,3,1,3
4,3,4,1,4,-1,4,0,-1,4,1,4,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,16,2,14,-1,-1,6,-1,-1,1995,3,6,2,1904
1996,11,6,20,13,13,0,0,2,1996,2,3,4,1905
1997,9,5,4,19,7,4,-1,-1,1997,1,0,5,1906
1998,10,16,4,-1,-1,6,4,3,1998,1,2,5,1907


In [25]:

targets =  torch.tensor(df['Average_Salary'].values, dtype=torch.int64)

targets

tensor([10, 11, 17,  ..., 15,  7,  7])

In [26]:
dataset =  TensorDataset(inputs, targets)

In [27]:
dataset[0]

(tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.]),
 tensor(10))

In [28]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [29]:
torch.manual_seed(0)
next(iter(dataloader))

[tensor([[ 1.7000e+01,  1.1000e+01,  6.0000e+00, -1.0000e+00, -1.0000e+00,
           7.0000e+00,  3.0000e+00,  2.0000e+00,  1.2260e+03,  2.0000e+00,
           2.0000e+00,  5.0000e+00,  1.1920e+03],
         [ 2.0000e+00,  1.6000e+01,  7.0000e+00,  3.0000e+00, -1.0000e+00,
           3.0000e+00,  2.0000e+00,  5.0000e+00,  5.0900e+02,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  5.0300e+02],
         [ 9.0000e+00,  2.1000e+01,  1.5000e+01,  0.0000e+00, -1.0000e+00,
           6.0000e+00,  4.0000e+00,  5.0000e+00,  1.6310e+03,  0.0000e+00,
           1.0000e+00,  6.0000e+00,  1.5690e+03],
         [ 2.0000e+00,  6.0000e+00,  1.9000e+01, -1.0000e+00, -1.0000e+00,
           4.0000e+00,  5.0000e+00,  0.0000e+00,  1.1320e+03,  1.0000e+00,
           6.0000e+00,  5.0000e+00,  1.1040e+03]]),
 tensor([11,  7, 13,  5])]

In [30]:
# [THIS IS READ-ONLY]
class LinearClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(13, 3)
    def forward(self, x):
        # x ~ (batch_size, 4)
        return self.fc(x)


In [31]:
torchinfo.summary(LinearClassifier())

Layer (type:depth-idx)                   Param #
LinearClassifier                         --
├─Linear: 1-1                            42
Total params: 42
Trainable params: 42
Non-trainable params: 0

In [32]:
torch.manual_seed(0)
x, targets = next(iter(dataloader))
model = LinearClassifier()
y = model(x)
y

tensor([[-196.0265,  -48.9636,  205.3186],
        [ -78.5724,  -24.0096,   88.7119],
        [-260.0611,  -68.0360,  275.0341],
        [-180.1619,  -49.2741,  187.5451]], grad_fn=<AddmmBackward0>)

In [33]:
import torch.nn.functional as F
def loss_fn(y, targets):
    loss = F.cross_entropy(y, targets)
    return loss

In [34]:
torch.manual_seed(0)
x, targets = next(iter(dataloader))
model = LinearClassifier()
y = model(x)
with torch.no_grad():
    loss = loss_fn(y, targets)
loss

IndexError: Target 11 is out of bounds.

In [ ]:
targets